In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
# Initialize the Edge web driver (adjust the executable_path as needed)
driver = webdriver.Edge()

# Navigate to the TUM degree programs page
url = "https://www.google.com/maps/search/iconic+place+in+bali/"
driver.get(url)

In [7]:
# Initialize some variables
iteration = 0  # Counter for consecutive iterations with no new elements
num_scrolls = 0  # Total scrolls performed
scroll_pause_time = 3  # Time to wait between scrolls (adjust as needed)

# Create an ActionChains object
action = ActionChains(driver)

while True:
    # Get the current number of elements
    scroll = driver.find_elements(By.CLASS_NAME, "hfpxzc")
    
    # Scroll to the bottom of the page
    action.key_down(Keys.END).key_up(Keys.END).perform()
    
    # Wait for a short time to allow new elements to load (adjust as needed)
    time.sleep(scroll_pause_time)
    
    # Update the elements list
    new_scroll = driver.find_elements(By.CLASS_NAME, "hfpxzc")

    # Calculate the number of new elements found in this iteration
    new_elements_count = len(new_scroll) - len(scroll)
    
    # Check if new elements have been loaded
    if new_elements_count > 0:
        iteration = 0  # Reset the counter if new elements are found
    else:
        iteration += 1
    
    # Check for the termination condition
    if iteration > 2:  # No new elements found for 2 consecutive iterations
        break
    
    # Update the elements list for the next iteration
    scroll = new_scroll
    
    num_scrolls += 1
    print(f"Scrolled {num_scrolls} times. Found {new_elements_count} new elements in this iteration.")

# Scroll back to the top of the page
action.key_down(Keys.HOME).key_up(Keys.HOME).perform()

Scrolled 1 times. Found 5 new elements in this iteration.
Scrolled 2 times. Found 5 new elements in this iteration.
Scrolled 3 times. Found 5 new elements in this iteration.
Scrolled 4 times. Found 5 new elements in this iteration.
Scrolled 5 times. Found 5 new elements in this iteration.
Scrolled 6 times. Found 5 new elements in this iteration.
Scrolled 7 times. Found 5 new elements in this iteration.
Scrolled 8 times. Found 5 new elements in this iteration.
Scrolled 9 times. Found 5 new elements in this iteration.
Scrolled 10 times. Found 5 new elements in this iteration.
Scrolled 11 times. Found 3 new elements in this iteration.
Scrolled 12 times. Found 0 new elements in this iteration.
Scrolled 13 times. Found 0 new elements in this iteration.


In [8]:
# Wait for the search results to load
time.sleep(3)

# Find all destination elements on the page
destination_elements = driver.find_elements(By.XPATH, '//div[@class="Nv2PK THOPZb CpccDe "]')

# Create a list to store the collected data
destination_data = []

# Loop through each destination element
for destination in destination_elements:
    # Check if the element already has the desired class
    if "CdoAJb" not in destination.get_attribute("class"):
        # If it doesn't have the class, change it
        driver.execute_script('arguments[0].setAttribute("class", "Nv2PK THOPZb CpccDe CdoAJb");', destination)

    # Click on the modified element
    driver.execute_script("arguments[0].scrollIntoView(true);", destination)
    destination.click()

    try:
        time.sleep(3)
        # Collect data for the current destination
        name = driver.find_element(By.XPATH, '//h1[@class="DUwDvf lfPIob"]').text
        label = driver.find_element(By.XPATH, '//button[@class="DkEaL "]').text
        rating = driver.find_element(By.XPATH, '//span[@class="ceNzKf"]').get_attribute('aria-label').replace("stars", "").strip()
        num_review = driver.find_element(By.XPATH, '//button[@class="HHrUdb fontTitleSmall rqjGif"]/span').text.replace("reviews", "").strip().replace(',', '')
        description = driver.find_element(By.XPATH, '//div[@class="PYvSYb "]').text
        location = driver.find_element(By.XPATH, '//div[@class="Io6YTe fontBodyMedium kR99db "]').text

        # Create a dictionary to store the data for the current destination
        destination_info = {
            'Name': name,
            'Label': label,
            'Rating': rating,
            'Number of Reviews': num_review,
            'Description': description,
            'Location': location
        }
    except Exception as e:
        # Handle the case where data collection is not available
        print("Data collection for the current destination is not available.")
        # Assign "Not Available" to all data fields
        destination_info = {
            'Name': 'Not Available',
            'Label': 'Not Available',
            'Rating': '0',
            'Number of Reviews': '0',
            'Description': 'Not Available',
            'Location': 'Not Available'
        }

    # Add the current destination's data to the list
    destination_data.append(destination_info)

# Print the collected data for all destinations
for data in destination_data:
    print(data)

Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data collection for the current destination is not available.
Data col

Save data to MySQL

In [16]:
from src.db.db_connection import establish_connection, create_cursor, create_destination_table, insert_destination_data

# Establish a database connection
connection = establish_connection()

# Create a cursor
cursor = create_cursor(connection)

# Create the destination table (if it doesn't exist)
create_destination_table(cursor)

# Loop through your destination data and insert it into the database
for data in destination_data:
    insert_destination_data(cursor, data)

# Commit changes and close the cursor and connection
connection.commit()
cursor.close()
connection.close()